<a id='jumpspot1'></a>
# Jump here

# MIMIC-FHIR Tutorial
Using Pathling, analyze the mimic-fhir resources

In [ ]:
from pathlib import Path
import requests
import json
import matplotlib.pyplot as plt
import pandas as pd

from fhirclient.models.parameters import Parameters, ParametersParameter
from py_mimic_fhir.lookup import MIMIC_FHIR_RESOURCES

In [ ]:
import_folder = 'file:///usr/share/staging'
server = 'http://localhost:8080/fhir'

### Upload resources to Pathling

In [ ]:
def generate_import_parameters(import_folder, profile, resource, mode):
    param_resource = Parameters()

    param_resource_type = ParametersParameter()
    param_resource_type.name= 'resourceType'
    param_resource_type.valueCode = resource

    param_url = {}
    param_url['name'] = 'url'
    param_url['valueUrl'] = f'{import_folder}/{profile}.ndjson'

    param_mode = ParametersParameter()
    param_mode.name= 'mode'
    param_mode.valueCode = mode

    param_source = ParametersParameter()
    param_source.name = 'source'
    param_source.part = [param_resource_type, param_url, param_mode]
    param_resource.parameter = [param_source]
    
    return param_resource.as_json()

In [ ]:
def post_import_ndjson(server, param):
    url = f'{server}/$import'

    resp = requests.post(url,  json = param, headers={"Content-Type": "application/fhir+json"} )
    return resp 

In [ ]:
mode = 'merge' # overwrite for fresh load (but not really since need to merge Observations not overwrite)

for profile, resource in MIMIC_FHIR_RESOURCES.items():
    if (resource == 'Observation') and (profile != 'ObservationChartevents'):
        param = generate_import_parameters(import_folder, profile, resource, mode)
        resp = post_import_ndjson(server, param)
        print(f"{profile}: {resp.json()['issue'][0]['diagnostics']}")

### Search Resources

In [ ]:
resource_type = 'Patient'
filter_path = "gender='male'"
count = 10

url = f'{server}/{resource_type}?_query=fhirPath&filter={filter_path}&_count={count}' 

resp = requests.get(url,  headers={"Content-Type": "application/fhir+json"} )
resp.json()


### Aggregate Resources

In [ ]:
def get_aggregate(resource_type, element_path, filter_path=None):
    server = 'http://localhost:8080/fhir'    
    url = f'{server}/{resource_type}/$aggregate?aggregation=count()&grouping={element_path}'

    if filter_path is not None:
        url = f'{url}&filter={filter_path}'
    

    resp = requests.get(url, headers={"Content-Type": "application/fhir+json"} )
    return resp.json()


In [ ]:
def plot_aggregate(resp, title, limit):
     parameters = resp['parameter']
     list_label= []
     list_value = []
     for parameter in parameters:
          if (len(parameter['part'][0]) == 2):
               label_val = list(parameter['part'][0].values())[1]
          else:
               label_val = 'WITHOUT'
          list_label.append(label_val)
          list_value.append(parameter['part'][1]['valueUnsignedInt'])

     df = pd.DataFrame({'label': list_label, 'value': list_value})
     df_sorted = df.sort_values(by=['value'], ascending=False).iloc[0:limit] 
     plt.bar(df_sorted['label'], df_sorted['value'])
     plt.xticks(rotation=90)
     plt.title(title)

In [ ]:
len(resp['parameter'][0]['part'][0])

In [ ]:
parameters = resp['parameter']
list_label= []
list_value = []
for parameter in parameters:
    label_val = list(parameter['part'][0].values())[1]
    list_label.append(label_val)

    list_value.append(parameter['part'][1]['valueUnsignedInt'])

df = pd.DataFrame({'label': list_label, 'value': list_value})
df_sorted = df.sort_values(by=['value'], ascending=False).iloc[0:10] 
plt.bar(df_sorted['label'], df_sorted['value'])
plt.xticks(rotation=45)
plt.title(element_path)

In [ ]:
resource_type = 'Condition'
element_path = 'code.coding.display'
title = f'{resource_type}: {element_path}'
limit = 10

resp = get_aggregate(resource_type, element_path)
plot_aggregate(resp, title, limit)

In [ ]:
resource_type = 'Patient'
element_path = 'gender'
limit=10

resp = get_aggregate(resource_type, element_path)
plot_aggregate(resp, element_path, limit)

In [ ]:
resource_type = 'Patient'
#element_path = "reverseResolve(Condition.subject).code.coding.where(subsumedBy(http://fhir.mimic.mit.edu/CodeSystem/diagnosis-icd9|4019))"
element_path = "reverseResolve(Condition.subject).code.coding.where(subsumedBy(http://fhir.mimic.mit.edu/CodeSystem/diagnosis-icd9|4019)).code"
filter_path="gender='male'"
title = f'{resource_type}: Males with hypertension'
limit = 10

resp = get_aggregate(resource_type, element_path, filter_path)
resp
plot_aggregate(resp, title, limit)

### Aggregation Data Integrity
- are there vital signs for all patients?
- other questions...

In [ ]:
# chartevents too large for pathling, crashes it currently (partitions out data by ~150MB, and chartevents is ~630MB)
resource_type = 'Patient'
element_path = "reverseResolve(Observation.subject).code.coding.where(subsumedBy(http://fhir.mimic.mit.edu/CodeSystem/chartevents-d-items|220045)).code"
title = f'{resource_type}: Patients with heartrate measured'
limit = 10

resp = get_aggregate(resource_type, element_path)
resp
plot_aggregate(resp, title, limit)

In [ ]:
resp

In [ ]:
resource_type = 'Patient'
element_path = "reverseResolve(MedicationAdministration.subject).medicationCodeableConcept.coding.where(subsumedBy(http://fhir.mimic.mit.edu/CodeSystem/medication-formulary-drug-cd|HEPA5I)).code"
title = f'{resource_type}: Heparin meds'
limit = 10

resp = get_aggregate(resource_type, element_path)
resp
plot_aggregate(resp, title, limit)

In [ ]:
resource_type = 'MedicationAdministration'
element_path = "medicationCodeableConcept.coding.code"
filter_path = "meta.where(profile.first()='http://fhir.mimic.mit.edu/StructureDefinition/mimic-medication-administration').empty().not()"
title = f'{resource_type}: Top EMAR medication administered'
limit = 10

resp = get_aggregate(resource_type, element_path, filter_path)
resp
plot_aggregate(resp, title, limit)

In [ ]:
resource_type = 'MedicationAdministration'
element_path = "medicationCodeableConcept.coding.display"
filter_path = "meta.where(profile.first()='http://fhir.mimic.mit.edu/StructureDefinition/mimic-medication-administration-icu').empty().not()"
title = f'{resource_type}: Top ICU medication administered'
limit = 10

resp = get_aggregate(resource_type, element_path, filter_path)
resp
plot_aggregate(resp, title, limit)

In [ ]:
# patients with hypertension, what are the top meds?
resource_type = 'Patient'
element_path = "reverseResolve(MedicationAdministration.subject).medicationCodeableConcept.coding.display"
filter_path = "reverseResolve(Condition.subject).code.coding.where(subsumedBy(http://fhir.mimic.mit.edu/CodeSystem/diagnosis-icd9|4019)).empty().not()"
title = f'{resource_type}: Top medication administered for Hypertension patients'
limit = 10

resp = get_aggregate(resource_type, element_path, filter_path)
resp
plot_aggregate(resp, title, limit)

In [ ]:
# try observation again... very large cause of chartevents... (could remove chartevents for demo so can at least look at others)
resource_type = 'Observation'
element_path = "code.coding.display"
filter_path = "meta.where(profile.first()='http://fhir.mimic.mit.edu/StructureDefinition/mimic-observation-labevents').empty().not()"
title = f'{resource_type}: Top observation labs'
limit = 10

resp = get_aggregate(resource_type, element_path, filter_path)
resp
plot_aggregate(resp, title, limit)

In [ ]:
# top labs for patients with hypertension
resource_type = 'Patient'
element_path = "reverseResolve(Observation.subject).code.coding.where(system.first()='http://fhir.mimic.mit.edu/CodeSystem/d-labitems').display"
filter_path = "reverseResolve(Condition.subject).code.coding.where(subsumedBy(http://fhir.mimic.mit.edu/CodeSystem/diagnosis-icd9|4019)).empty().not()"
title = f'{resource_type}: Top labs runs for Hypertension patients'
limit = 20

resp = get_aggregate(resource_type, element_path, filter_path)
resp
plot_aggregate(resp, title, limit)

In [ ]:
# top conditions for patients
resource_type = 'Patient'
element_path = "reverseResolve(Condition.subject).code.coding.display"
#filter_path = "reverseResolve(Condition.subject).code.coding.where(subsumedBy(http://fhir.mimic.mit.edu/CodeSystem/diagnosis-icd9|4019)).empty().not()"
title = f'{resource_type}: Top conditions in hospital'
limit = 10

resp = get_aggregate(resource_type, element_path)
resp
plot_aggregate(resp, title, limit)

### Extract resource table

In [ ]:
def get_extract(server, resource_type, columns, limit):      
    url = f'{server}/{resource_type}/$extract?'

    for column in columns:
        url = f'{url}column={column}&'
    
    url = f'{url}limit={limit}'
    resp = requests.get(url, headers={"Content-Type": "application/fhir+json"} )
    return resp.json()

In [ ]:
server = 'http://localhost:8080/fhir'  
resource_type = 'Patient'
column1 = 'gender'
column2 = 'birthDate'
columns = [column1, column2]
title = f'{resource_type}: {element_path}'
limit = 10

resp = get_extract(server, resource_type, columns, limit)
resp

[jump to top](#jumpspot1)